# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846868117107                   -0.70    4.5         
  2   -7.852318884429       -2.26       -1.53    1.0   17.5ms
  3   -7.852615671771       -3.53       -2.56    1.5   18.6ms
  4   -7.852646058815       -4.52       -2.90    2.5   23.2ms
  5   -7.852646523801       -6.33       -3.19    1.2   18.1ms
  6   -7.852646679805       -6.81       -4.00    1.0   17.5ms
  7   -7.852646686361       -8.18       -5.28    1.5   19.6ms
  8   -7.852646686723       -9.44       -5.44    2.2   23.4ms
  9   -7.852646686730      -11.20       -6.34    1.0   17.7ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846961354659                   -0.70           4.5         
  2   -7.852553767049       -2.25       -1.64   0.80    2.0    216ms
  3   -7.852639097248       -4.07       -2.75   0.80    1.0   17.5ms
  4   -7.852646505382       -5.13       -3.34   0.80    2.0   20.9ms
  5   -7.852646679958       -6.76       -4.22   0.80    1.5   18.2ms
  6   -7.852646686577       -8.18       -4.78   0.80    1.8   26.5ms
  7   -7.852646686720       -9.84       -5.58   0.80    1.2   30.7ms
  8   -7.852646686730      -11.03       -6.46   0.80    2.0   22.6ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.429371e+01     3.961312e+00
 * time: 0.047467947006225586
     1     1.252168e+00     1.877055e+00
 * time: 0.22797393798828125
     2    -1.740138e+00     2.193008e+00
 * time: 0.24622678756713867
     3    -3.742473e+00     2.056474e+00
 * time: 0.2724928855895996
     4    -4.998269e+00     1.778979e+00
 * time: 0.2986898422241211
     5    -6.676719e+00     1.133724e+00
 * time: 0.3247859477996826
     6    -7.413169e+00     4.294971e-01
 * time: 0.40535783767700195
     7    -7.695244e+00     2.752075e-01
 * time: 0.4235858917236328
     8    -7.792410e+00     9.546171e-02
 * time: 0.4416079521179199
     9    -7.828205e+00     1.248922e-01
 * time: 0.4594919681549072
    10    -7.839770e+00     4.840447e-02
 * time: 0.47742795944213867
    11    -7.844274e+00     5.118664e-02
 * time: 0.49525880813598633
    12    -7.845322e+00     5.278988e-02
 * time: 0.5130789279937744
    13    -7.845370e+00     6.355834e-02
 * time: 0.538

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846896005646                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645922247                   -1.64         
  2   -7.852646686730       -6.12       -3.72    1.52s
  3   -7.852646686730      -13.32       -7.25    121ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 6.875795218809306e-7
|ρ_newton - ρ_scfv| = 9.109948393482596e-8
|ρ_newton - ρ_dm|   = 9.90880265490772e-10
